# CSI 4142 Data Science 
## Assignment 2 - Data Cleaning

### Identification

Name: Eli Wynn<br/>
Student Number: 300248135

Name: Jack Snelgrove<br/>
Student Number: 300247435


Our datasets have been uploaded from the public repository:

- [github.com/eli-wynn/Datasets](https://github.com/eli-wynn/Datasets)

Imports:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

### Importing Datasets

In [ ]:
netflix  = "https://raw.githubusercontent.com/eli-wynn/Datasets/refs/heads/main/netflix_titles.csv"
netflixData = pd.read_csv(netflix)
startup = "https://raw.githubusercontent.com/eli-wynn/Datasets/refs/heads/main/startup.csv"
startupData = pd.read_csv(startup)

### Clean Data Checker

#### Data Type Error

A data type error occurs when the the data entered into a column doesnt match the data type assigned to that column. There are zero datatype errors in the Netflix dataset


#### Parameters

In [ ]:
intCol = ['release_year']
stringCols = ['show_id', 'type', 'title', 'director', 'cast', 'country', 'rating', 'duration', 'listed_in', 'description']

#### Checker Code

In [ ]:
for col in intCol:
    netflixData[col] = pd.to_numeric(netflixData[col], errors='coerce')

for col in stringCols:
    invalid_strings = netflixData[~netflixData[col].astype(str).apply(lambda x: isinstance(x, str))]
    if not invalid_strings.empty:
        print(f"\nPossible non-string values in '{col}':\n", invalid_strings.head(5))

#### Findings

In [ ]:
# Find rows where conversion resulted in NaN (potential type errors)
type_errors = netflixData[netflixData[intCol].isna().any(axis=1)]
print("Possible data type errors:\n", type_errors)

#### Range Errors

Searches for errors where the data is outside acceptable range (e.g. season -1 or release date prior to 1930)

#### Parameters

In [ ]:
releaseParam = [1925, 2025]
durationParams = [0, 300] #split on space and make sure first item in array is >0 <300
dateAdded = [2007, 2025] #just look at year 

#### Checker Code

In [ ]:
releaseErrors = netflixData[(netflixData['release_year'] < releaseParam[0]) | (netflixData['release_year'] > releaseParam[1])]

netflixData['duration_split'] = netflixData['duration'].str.split(" ").str[0]  # Extract number part
netflixData['duration_split'] = pd.to_numeric(netflixData['duration_split'], errors='coerce')  # Convert to int
# Identify invalid durations
durationErrors = netflixData[(netflixData['duration_split'] <= durationParams[0]) | (netflixData['duration_split'] >= durationParams[1])]

netflixData['date_added'] = pd.to_datetime(netflixData['date_added'], errors='coerce')
netflixData['year_added'] = netflixData['date_added'].dt.year #take just year value, other date errors will be caught in format error below
# Identify invalid date_added values
dateAddedErrors = netflixData[(netflixData['year_added'] < dateAdded[0]) | (netflixData['year_added'] > dateAdded[1])]

#### Findings

Only one duration error, a black mirror episode longer than 300 minutes, don't know whether it is just an abnormally long episode outside my set parameters or needs to be cleaned up

In [ ]:
print("\nRelease Year Errors:\n", releaseErrors[['title', 'release_year']].head(5))
print("\nDuration Errors:\n", durationErrors[['show_id', 'title', 'duration', 'duration_split']].head(5))
print("\nDate Added Errors:\n", dateAddedErrors[['title', 'date_added', 'year_added']].head(5))

#### Format Errors

Checks for errors with the formatting of the data, e.g. date being DD-MM-YYYY instead of YYYY first

#### Parameters

In [ ]:
dateCol = ['date_added'] #make sure date is correct format
showCol = ['show_id'] #make sure it is s### format
durationCol = ['duration'] #make sure duration is number followed by either "min" or "Season" or "Seasons"

#### Checker Code

In [ ]:
for col in dateCol:
    netflixData[col] = pd.to_datetime(netflixData[col], errors='coerce')
invalid_dates = netflixData[netflixData[dateCol].isna().any(axis=1)]

brokenID = netflixData[~netflixData['show_id'].astype(str).str.match(r"^s\d{1,4}$", na=False)] #finds all ids that don't match format of s followed by 1-4 #'s
brokenDuration = netflixData[~netflixData['duration'].astype(str).str.match(r"^\d+\s(min|Season|Seasons)$", na=False)] #finds all id's that don't match digits then space then seasons, season or min

#### Findings

Louis C.K. titles have the durations in the ratings column for some reason - formatting is technically correct though
<br> No show ID errors
<br> There are a few date formatting errors

In [ ]:
print("\nPossible date format errors:\n", invalid_dates[['title', 'date_added']].head(5))
print("\nShow ID Format Errors:\n", brokenID[['show_id', 'title']].head(5))
print("\nDuration Format Errors:\n", brokenDuration[['duration', 'title']].head(5)) #Louis C.k. durations are in rating column for some reason

#### Consistency Errors

a type of logical check that ensures data is entered in a
logically consistent manner. In this data a consistensy error could be checking that movies all have their duration in minutes and shows in seasons

#### Parameters

In [ ]:
#parameters changed below by updating movie or tv show

#### Checker Code

In [ ]:
movieErr = netflixData[(netflixData['type'] == 'Movie')&~netflixData['duration'].astype(str).str.match(r"^\d+\smin$", na=False)]
tvErr = netflixData[(netflixData['type'] == 'TV Show')&~netflixData['duration'].astype(str).str.match(r"^\d+\s(Season|Seasons)$", na=False)] #match all tv show types and regex to ensure consistent season/seasons format

#### Findings

No errors in the consistency just the same duration absences discovered above in the louis C.K. stuff

In [ ]:
print("\nIncosistent Movie Durations: \n", movieErr[['title', 'type', 'duration']].head(5))
print("\nIncosistent Show Durations: \n", tvErr[['title', 'type', 'duration']].head(5))


#### Uniqueness Errors

Ensure that there are no duplicate values, The title column is the only applicable

#### Parameters

In [ ]:
uniquenessParams = ['show_id', 'title', 'release_year']

#### Checker Code

In [ ]:
dupeShowID = netflixData[netflixData.duplicated(subset=[uniquenessParams[0]], keep=False)] #showID duplicate

#Checking for duplicate titles isn't valid because two movies can have the same names so need to compare with release year as well
dupeTitle = netflixData[netflixData.duplicated(subset=[uniquenessParams[1], uniquenessParams[2]], keep=False)]

#### Findings

In [ ]:
print("\nDuplicate Show IDs: \n", dupeShowID[[uniquenessParams[0]].head(5)])
print("\nDuplicate titles: \n", dupeTitle[[uniquenessParams[1], uniquenessParams[2]].head(5)])

#### Presence Errors

A presence error is when a mandatory value is left blank, in this case you could argue that every column is necessary but title, id and type are the most important

#### Parameters

In [ ]:
mandatoryVals = ['show_id', 'title', 'release_year', 'type', 'duration']

#### Checker Code

In [ ]:
missingValues = netflixData[netflixData[[mandatoryVals[0], mandatoryVals[1], mandatoryVals[2], mandatoryVals[3], mandatoryVals[4]]].isnull().any(axis=1)]


#### Findings

In [ ]:
print("\nMissing mandatory values: \n", missingValues[['show_id', 'title', 
                                                'release_year', 'type', 'duration']].head(5))

#### Length Errors

Errors in which the length of a parameter is different then the norm. e.g. A title that is 2000 characters long is very likely a length error and an incorrectly input piece of data because 2000 characters is unreasonable to say

#### Parameters

In [ ]:
titleSizeParam = [1,200]
relYearParam = [4]

#### Checker Code

In [ ]:
titleLen = netflixData[(netflixData['title'].str.len() < titleSizeParam[0]) | (netflixData['title'].str.len() > titleSizeParam[1])]
relYearLen = netflixData[(netflixData['release_year'].str.len() == releaseParam[0])]

#### Findings

In [ ]:
print("\nErroneous Title Lengths: \n", titleLen['title', 'show_id'].head(5))

#### Lookup Errors

Errors which are caused by a value being outside the correct set of values. For example, having a made up month in a month of release. In this dataset lookup errors can be present in the country value, rating or type columns

#### Parameters

In [ ]:
#Country dictionary sourced from: https://stackoverflow.com/questions/41245330/check-if-a-country-entered-is-one-of-the-countries-of-the-world
Country = [
    ('US', 'United States'),
    ('AF', 'Afghanistan'),
    ('AL', 'Albania'),
    ('DZ', 'Algeria'),
    ('AS', 'American Samoa'),
    ('AD', 'Andorra'),
    ('AO', 'Angola'),
    ('AI', 'Anguilla'),
    ('AQ', 'Antarctica'),
    ('AG', 'Antigua And Barbuda'),
    ('AR', 'Argentina'),
    ('AM', 'Armenia'),
    ('AW', 'Aruba'),
    ('AU', 'Australia'),
    ('AT', 'Austria'),
    ('AZ', 'Azerbaijan'),
    ('BS', 'Bahamas'),
    ('BH', 'Bahrain'),
    ('BD', 'Bangladesh'),
    ('BB', 'Barbados'),
    ('BY', 'Belarus'),
    ('BE', 'Belgium'),
    ('BZ', 'Belize'),
    ('BJ', 'Benin'),
    ('BM', 'Bermuda'),
    ('BT', 'Bhutan'),
    ('BO', 'Bolivia'),
    ('BA', 'Bosnia And Herzegowina'),
    ('BW', 'Botswana'),
    ('BV', 'Bouvet Island'),
    ('BR', 'Brazil'),
    ('BN', 'Brunei Darussalam'),
    ('BG', 'Bulgaria'),
    ('BF', 'Burkina Faso'),
    ('BI', 'Burundi'),
    ('KH', 'Cambodia'),
    ('CM', 'Cameroon'),
    ('CA', 'Canada'),
    ('CV', 'Cape Verde'),
    ('KY', 'Cayman Islands'),
    ('CF', 'Central African Rep'),
    ('TD', 'Chad'),
    ('CL', 'Chile'),
    ('CN', 'China'),
    ('CX', 'Christmas Island'),
    ('CC', 'Cocos Islands'),
    ('CO', 'Colombia'),
    ('KM', 'Comoros'),
    ('CG', 'Congo'),
    ('CK', 'Cook Islands'),
    ('CR', 'Costa Rica'),
    ('CI', 'Cote D`ivoire'),
    ('HR', 'Croatia'),
    ('CU', 'Cuba'),
    ('CY', 'Cyprus'),
    ('CZ', 'Czech Republic'),
    ('DK', 'Denmark'),
    ('DJ', 'Djibouti'),
    ('DM', 'Dominica'),
    ('DO', 'Dominican Republic'),
    ('TP', 'East Timor'),
    ('EC', 'Ecuador'),
    ('EG', 'Egypt'),
    ('SV', 'El Salvador'),
    ('GQ', 'Equatorial Guinea'),
    ('ER', 'Eritrea'),
    ('EE', 'Estonia'),
    ('ET', 'Ethiopia'),
    ('FK', 'Falkland Islands (Malvinas)'),
    ('FO', 'Faroe Islands'),
    ('FJ', 'Fiji'),
    ('FI', 'Finland'),
    ('FR', 'France'),
    ('GF', 'French Guiana'),
    ('PF', 'French Polynesia'),
    ('TF', 'French S. Territories'),
    ('GA', 'Gabon'),
    ('GM', 'Gambia'),
    ('GE', 'Georgia'),
    ('DE', 'Germany'),
    ('GH', 'Ghana'),
    ('GI', 'Gibraltar'),
    ('GR', 'Greece'),
    ('GL', 'Greenland'),
    ('GD', 'Grenada'),
    ('GP', 'Guadeloupe'),
    ('GU', 'Guam'),
    ('GT', 'Guatemala'),
    ('GN', 'Guinea'),
    ('GW', 'Guinea-bissau'),
    ('GY', 'Guyana'),
    ('HT', 'Haiti'),
    ('HN', 'Honduras'),
    ('HK', 'Hong Kong'),
    ('HU', 'Hungary'),
    ('IS', 'Iceland'),
    ('IN', 'India'),
    ('ID', 'Indonesia'),
    ('IR', 'Iran'),
    ('IQ', 'Iraq'),
    ('IE', 'Ireland'),
    ('IL', 'Israel'),
    ('IT', 'Italy'),
    ('JM', 'Jamaica'),
    ('JP', 'Japan'),
    ('JO', 'Jordan'),
    ('KZ', 'Kazakhstan'),
    ('KE', 'Kenya'),
    ('KI', 'Kiribati'),
    ('KP', 'Korea (North)'),
    ('KR', 'Korea (South)'),
    ('KW', 'Kuwait'),
    ('KG', 'Kyrgyzstan'),
    ('LA', 'Laos'),
    ('LV', 'Latvia'),
    ('LB', 'Lebanon'),
    ('LS', 'Lesotho'),
    ('LR', 'Liberia'),
    ('LY', 'Libya'),
    ('LI', 'Liechtenstein'),
    ('LT', 'Lithuania'),
    ('LU', 'Luxembourg'),
    ('MO', 'Macau'),
    ('MK', 'Macedonia'),
    ('MG', 'Madagascar'),
    ('MW', 'Malawi'),
    ('MY', 'Malaysia'),
    ('MV', 'Maldives'),
    ('ML', 'Mali'),
    ('MT', 'Malta'),
    ('MH', 'Marshall Islands'),
    ('MQ', 'Martinique'),
    ('MR', 'Mauritania'),
    ('MU', 'Mauritius'),
    ('YT', 'Mayotte'),
    ('MX', 'Mexico'),
    ('FM', 'Micronesia'),
    ('MD', 'Moldova'),
    ('MC', 'Monaco'),
    ('MN', 'Mongolia'),
    ('MS', 'Montserrat'),
    ('MA', 'Morocco'),
    ('MZ', 'Mozambique'),
    ('MM', 'Myanmar'),
    ('NA', 'Namibia'),
    ('NR', 'Nauru'),
    ('NP', 'Nepal'),
    ('NL', 'Netherlands'),
    ('AN', 'Netherlands Antilles'),
    ('NC', 'New Caledonia'),
    ('NZ', 'New Zealand'),
    ('NI', 'Nicaragua'),
    ('NE', 'Niger'),
    ('NG', 'Nigeria'),
    ('NU', 'Niue'),
    ('NF', 'Norfolk Island'),
    ('MP', 'Northern Mariana Islands'),
    ('NO', 'Norway'),
    ('OM', 'Oman'),
    ('PK', 'Pakistan'),
    ('PW', 'Palau'),
    ('PA', 'Panama'),
    ('PG', 'Papua New Guinea'),
    ('PY', 'Paraguay'),
    ('PE', 'Peru'),
    ('PH', 'Philippines'),
    ('PN', 'Pitcairn'),
    ('PL', 'Poland'),
    ('PT', 'Portugal'),
    ('PR', 'Puerto Rico'),
    ('QA', 'Qatar'),
    ('RE', 'Reunion'),
    ('RO', 'Romania'),
    ('RU', 'Russian Federation'),
    ('RW', 'Rwanda'),
    ('KN', 'Saint Kitts And Nevis'),
    ('LC', 'Saint Lucia'),
    ('VC', 'St Vincent/Grenadines'),
    ('WS', 'Samoa'),
    ('SM', 'San Marino'),
    ('ST', 'Sao Tome'),
    ('SA', 'Saudi Arabia'),
    ('SN', 'Senegal'),
    ('SC', 'Seychelles'),
    ('SL', 'Sierra Leone'),
    ('SG', 'Singapore'),
    ('SK', 'Slovakia'),
    ('SI', 'Slovenia'),
    ('SB', 'Solomon Islands'),
    ('SO', 'Somalia'),
    ('ZA', 'South Africa'),
    ('ES', 'Spain'),
    ('LK', 'Sri Lanka'),
    ('SH', 'St. Helena'),
    ('PM', 'St.Pierre'),
    ('SD', 'Sudan'),
    ('SR', 'Suriname'),
    ('SZ', 'Swaziland'),
    ('SE', 'Sweden'),
    ('CH', 'Switzerland'),
    ('SY', 'Syrian Arab Republic'),
    ('TW', 'Taiwan'),
    ('TJ', 'Tajikistan'),
    ('TZ', 'Tanzania'),
    ('TH', 'Thailand'),
    ('TG', 'Togo'),
    ('TK', 'Tokelau'),
    ('TO', 'Tonga'),
    ('TT', 'Trinidad And Tobago'),
    ('TN', 'Tunisia'),
    ('TR', 'Turkey'),
    ('TM', 'Turkmenistan'),
    ('TV', 'Tuvalu'),
    ('UG', 'Uganda'),
    ('UA', 'Ukraine'),
    ('AE', 'United Arab Emirates'),
    ('UK', 'United Kingdom'),
    ('UY', 'Uruguay'),
    ('UZ', 'Uzbekistan'),
    ('VU', 'Vanuatu'),
    ('VA', 'Vatican City State'),
    ('VE', 'Venezuela'),
    ('VN', 'Viet Nam'),
    ('VG', 'Virgin Islands (British)'),
    ('VI', 'Virgin Islands (U.S.)'),
    ('EH', 'Western Sahara'),
    ('YE', 'Yemen'),
    ('YU', 'Yugoslavia'),
    ('ZR', 'Zaire'),
    ('ZM', 'Zambia'),
    ('ZW', 'Zimbabwe')
]

validCountries = {name for code, name in Country}
validRatings = {"G", "PG", "PG-13", "R", "NC-17", "TV-Y", "TV-Y7", "TV-G", "TV-PG", "TV-14", "TV-MA"}

#### Checker Code

In [ ]:
countryError = netflixData[netflixData['country'].notna() & ~netflixData['country'].isin(validCountries)]
ratingError = netflixData[netflixData['rating'].notna() & ~netflixData['rating'].isin(validRatings)]


#### Findings

There are many country lookup errors in the data because my check doesn't account for multiple values inside the country column. There are still errors with the wrong stuff being present in the wrong columns in the louis CK ones as well

In [ ]:
print("\nCountry error: \n", countryError[['title', 'show_id', 'country']].head(5))
print("\nRating error: \n", ratingError[['title', 'show_id', 'rating']].head(5))

#### Exact Duplicate Errors
Exact duplicates occur when multiple rows in the dataset contain the exact same values 
for all attributes


#### Parameters

In [ ]:
duplicate_subset = ['title', 'show_id', 'country']  # Columns to check for exact duplicates


#### Checker Code

In [ ]:
exactDuplicates = netflixData[netflixData.duplicated(subset=duplicate_subset)]

#### Findings

There are no exact duplicates in the dataset

In [ ]:
print("\nExact Duplicate Errors: \n", exactDuplicates[duplicate_subset].head(5))

#### Near Duplicate Errors

Near duplicates are rows that are very similar but have slight differences. This may 
occur due to typos, inconsistent data formatting, or variations in missing values. 

#### Parameters

In [ ]:
#I decided to assume that the titles were correct and am using it as a key to look for near duplicates in comparison
#i.e. duplicates where title matches but other columns differ
titleCol = 'title'
comparisonCols = ['show_id', 'country', 'rating', 'director', 'cast', 'release_year']


#### Checker Code

In [ ]:
nearDuplicates = netflixData[netflixData.duplicated(subset=[titleCol], keep=False)]  # Find rows with duplicate titles

nearDuplicateErrors = nearDuplicates[nearDuplicates.apply(
    lambda row: any(row[comparisonCols] != nearDuplicates[comparisonCols].iloc[0]), #look for differences in comparison columns
    axis=1)]


#### Findings

There are no near duplicates with the same title but different values in the other columns

In [ ]:
print("\nNear Duplicate Errors: \n", nearDuplicateErrors[comparisonCols + [titleCol]].head(5))

### Imputation

#### Test #1

- a) Funding Rounds 
- b) 